<a href="https://colab.research.google.com/github/Ninjia315/AttractingContributors-Favorite_Japanese_songs/blob/main/Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: install numpy

!pip install numpy pandas scipy


In [2]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip


--2025-01-26 14:13:06--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  24.8MB/s    in 0.2s    

2025-01-26 14:13:06 (24.8 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.ba

In [3]:
import pandas as pd
import numpy as np

# Path to your dataset in Colab environment
data_path = '/content/ml-100k/u.data'

# Load the data
# By default, u.data is tab-separated, so we specify delimiter = '\t'
df = pd.read_csv(data_path,
                 sep='\t',
                 names=['user_id','item_id','rating','timestamp'])

# Inspect the first 5 rows
df.head()


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

print("Train size:", len(train_data))
print("Test size:", len(test_data))



Train size: 80000
Test size: 20000


In [12]:
num_users = df['user_id'].nunique()
num_items = df['item_id'].nunique()

# Initialize an all-zero matrix
interaction_matrix = np.zeros((num_users, num_items))

# Fill in the matrix with training data ratings
for row in train_data.itertuples():
    # Shift the user_id and item_id by -1 so they start at 0
    u_idx = row.user_id - 1
    i_idx = row.item_id - 1
    interaction_matrix[u_idx, i_idx] = row.rating

# Compute user means
user_means = interaction_matrix.mean(axis=1, where=(interaction_matrix!=0))

# Create a centered matrix: rating - user_mean (only for non-zero entries)
centered_matrix = np.zeros_like(interaction_matrix)
for u_idx in range(num_users):
    for i_idx in range(num_items):
        if interaction_matrix[u_idx, i_idx] > 0:
            centered_matrix[u_idx, i_idx] = interaction_matrix[u_idx, i_idx] - user_means[u_idx]

# Dot product of the centered matrix
sim_matrix = centered_matrix.dot(centered_matrix.T)

# compute norms to transform to cosine similarity
norms = np.sqrt(np.sum(centered_matrix**2, axis=1))
sim_matrix_cosine = sim_matrix / (norms[:,None] * norms[None,:] + 1e-9)  # avoid division by zero

print("Similarity Matrix (Cosine Similarity):")
print(sim_matrix_cosine)


Similarity Matrix (Cosine Similarity):
[[ 1.          0.02677565 -0.00950719 ... -0.01009321 -0.0801034
   0.03587929]
 [ 0.02677565  1.          0.00152648 ...  0.02956556 -0.00552181
   0.04177763]
 [-0.00950719  0.00152648  1.         ...  0.00690497  0.04902516
  -0.00388377]
 ...
 [-0.01009321  0.02956556  0.00690497 ...  1.         -0.02778681
  -0.01014216]
 [-0.0801034  -0.00552181  0.04902516 ... -0.02778681  1.
   0.00985081]
 [ 0.03587929  0.04177763 -0.00388377 ... -0.01014216  0.00985081
   1.        ]]


In [13]:
#A function keeps only the top-k similarities. All other similarities get set to 0.
def truncate_sim_to_topk(sim_matrix, k):
    """
    For each user u, keep only the top-k entries in sim_matrix[u, :].
    All other entries become zero.
    """
    # Make a copy so we don't overwrite the original
    sim_topk = np.zeros_like(sim_matrix)

    num_users = sim_matrix.shape[0]
    for u in range(num_users):
        # Sort indices of similarities in descending order
        # (argsort by default is ascending, so we invert by [::-1])
        # This gives us the indices of the highest similarities.
        top_k_idx = np.argsort(sim_matrix[u, :])[::-1][:k]

        # Copy only the top-k similarities into sim_topk
        for idx in top_k_idx:
            sim_topk[u, idx] = sim_matrix[u, idx]

    return sim_topk

# predict ratings given truncated similarity matrix (top-k)
def predict_ratings(sim_topk, centered_matrix, user_means):
    """
    Compute predicted ratings given truncated similarity matrix (top-k),
    centered ratings, and each user's mean rating.
    """
    # Sum of absolute similarities for normalization
    sim_sums = np.abs(sim_topk).sum(axis=1, keepdims=True) + 1e-9  # avoid dividing by zero

    # Weighted sum in centered space
    rating_pred_centered = sim_topk.dot(centered_matrix) / sim_sums

    # Add user means back in to get final absolute rating predictions
    final_rating_prediction = rating_pred_centered + user_means[:, None]

    return final_rating_prediction


In [14]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def evaluate_rmse(final_rating_prediction, test_data):
    """
    Calculate RMSE by comparing the model predictions with actual ratings in test_data.
    test_data columns: ['user_id', 'item_id', 'rating']
    """
    test_users = test_data['user_id'].values - 1  # shift to 0-based
    test_items = test_data['item_id'].values - 1
    test_ratings = test_data['rating'].values

    preds = []
    for idx in range(len(test_data)):
        u = test_users[idx]
        i = test_items[idx]
        # Clip to the [1, 5] range (MovieLens scale)
        pred_rating = np.clip(final_rating_prediction[u, i], 1, 5)
        preds.append(pred_rating)

    rmse = sqrt(mean_squared_error(test_ratings, preds))
    return rmse


In [22]:


k_values = [5, 10, 20, 50, 100, 200, 500]
for k in k_values:
    print(f"\n=== Top-{k} neighbors ===")

    # 1) Truncate sim_matrix to top-k
    sim_topk = truncate_sim_to_topk(sim_matrix_cosine, k)

    # 2) Predict ratings
    final_pred = predict_ratings(sim_topk, centered_matrix, user_means)

    # 3) Evaluate
    rmse_value = evaluate_rmse(final_pred, test_data)
    print(f"Test RMSE @ k={k}: {rmse_value:.4f}")



=== Top-5 neighbors ===
Test RMSE @ k=5: 1.0147

=== Top-10 neighbors ===
Test RMSE @ k=10: 1.0054

=== Top-20 neighbors ===
Test RMSE @ k=20: 1.0001

=== Top-50 neighbors ===
Test RMSE @ k=50: 0.9975

=== Top-100 neighbors ===
Test RMSE @ k=100: 0.9994

=== Top-200 neighbors ===
Test RMSE @ k=200: 1.0026

=== Top-500 neighbors ===
Test RMSE @ k=500: 1.0085


k= 50 appears to give the best balance, which means only choose the top 50 most similar neighbors for every user.

It shows how user–based collaborative filtering can be optimized by carefully selecting how many  similar behaviour neighbors to use in movie-rating scenario.